## Create ONNX Preprocessor

In [1]:
import sys

!{sys.executable} --version

Python 3.6.13


In [2]:
!curl https://bootstrap.pypa.io/get-pip.py | python
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade setuptools

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2596k  100 2596k    0     0  74.5M      0 --:--:-- --:--:-- --:--:-- 74.5M
ERROR: This script does not work on Python 3.6 The minimum supported Python version is 3.7. Please use https://bootstrap.pypa.io/pip/3.6/get-pip.py instead.


In [3]:
!{sys.executable} -m pip install onnx==1.11.0
!{sys.executable} -m pip install onnxruntime==1.10.0

In [4]:
from onnx import helper, version_converter
from onnx import TensorProto
from onnx.defs import onnx_opset_version

import onnxruntime as rt
import onnx
import numpy as np

In [5]:
onnx_opset_version()

16

In [6]:
X = helper.make_tensor_value_info('X', TensorProto.FLOAT, [1, 3, None, None])
sizes = helper.make_tensor_value_info('sizes', TensorProto.INT64, [4])
n1 = onnx.helper.make_node(
    'Resize',
    inputs=['X', '', '', 'sizes'],
    outputs=['T1'],
    coordinate_transformation_mode='tf_crop_and_resize',
    name='n1'
)

scale = helper.make_tensor_value_info('scale', TensorProto.FLOAT, [3])
bias = helper.make_tensor_value_info('bias', TensorProto.FLOAT, [3])
mean = helper.make_tensor_value_info('mean', TensorProto.FLOAT, [3])
var = helper.make_tensor_value_info('var', TensorProto.FLOAT, [3])
n2 = onnx.helper.make_node(
    'BatchNormalization',
    inputs=['T1', 'scale', 'bias', 'mean', 'var'],
    outputs=['Y'],
    epsilon=0.0,
    momentum=0.0,
    name='n2'
)

Y = helper.make_tensor_value_info('Y', TensorProto.FLOAT, [1, 3, 224, 224])

# Create the graph (GraphProto)
graph_def = helper.make_graph(
    [n1, n2],
    'preprocessor',
    [X, sizes, scale, bias, mean, var],
    [Y]
)

In [7]:
# Create the model (ModelProto)
model_def = helper.make_model(graph_def, producer_name='onnx-example')
model_def

ir_version: 8
producer_name: "onnx-example"
graph {
  node {
    input: "X"
    input: ""
    input: ""
    input: "sizes"
    output: "T1"
    name: "n1"
    op_type: "Resize"
    attribute {
      name: "coordinate_transformation_mode"
      s: "tf_crop_and_resize"
      type: STRING
    }
  }
  node {
    input: "T1"
    input: "scale"
    input: "bias"
    input: "mean"
    input: "var"
    output: "Y"
    name: "n2"
    op_type: "BatchNormalization"
    attribute {
      name: "epsilon"
      f: 0.0
      type: FLOAT
    }
    attribute {
      name: "momentum"
      f: 0.0
      type: FLOAT
    }
  }
  name: "preprocessor"
  input {
    name: "X"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 1
          }
          dim {
            dim_value: 3
          }
          dim {
          }
          dim {
          }
        }
      }
    }
  }
  input {
    name: "sizes"
    type {
      tensor_type {
        elem_type: 7
 

In [8]:
from onnx import version_converter

converted_model = version_converter.convert_version(model_def, 15)
converted_model

ir_version: 8
producer_name: "onnx-example"
graph {
  node {
    input: "X"
    input: ""
    input: ""
    input: "sizes"
    output: "T1"
    name: "n1"
    op_type: "Resize"
    attribute {
      name: "coordinate_transformation_mode"
      s: "tf_crop_and_resize"
      type: STRING
    }
  }
  node {
    input: "T1"
    input: "scale"
    input: "bias"
    input: "mean"
    input: "var"
    output: "Y"
    name: "n2"
    op_type: "BatchNormalization"
    attribute {
      name: "epsilon"
      f: 0.0
      type: FLOAT
    }
    attribute {
      name: "momentum"
      f: 0.0
      type: FLOAT
    }
  }
  name: "preprocessor"
  input {
    name: "X"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 1
          }
          dim {
            dim_value: 3
          }
          dim {
          }
          dim {
          }
        }
      }
    }
  }
  input {
    name: "sizes"
    type {
      tensor_type {
        elem_type: 7
 

In [9]:
# model_def.ir_version = 7
# model_def.opset_import[0].version = 14

In [10]:
onnx.checker.check_model(converted_model)

In [11]:
onnx.save(converted_model, 'preprocessor.onnx')

## Validate with ONNX Runtime

In [12]:
preprocessor = rt.InferenceSession('preprocessor.onnx')

In [13]:
data = np.array(
    [[
        [
            [1, 2, 3, 4],
            [1, 6, 7, 8],
            [1, 10, 11, 12], 
        ],
        [
            [1, 2, 3, 4],
            [1, 6, 7, 8],
            [1, 10, 11, 12], 
        ],
        [
            [1, 2, 3, 4],
            [1, 6, 7, 8],
            [1, 10, 11, 12], 
        ]
    ]]
)

sizes = np.array([1, 3, 224, 224])
scale = np.array([1, 1, 1])
bias = np.array([0, 0, 0])
mean = np.array([0.485, 0.456, 0.406])
var = np.sqrt(np.array([0.229, 0.224, 0.225]))

In [14]:
var

array([0.47853944, 0.47328638, 0.47434165])

In [15]:
input_name = preprocessor.get_inputs()[0].name
size_name = preprocessor.get_inputs()[1].name
scale_name = preprocessor.get_inputs()[2].name
bias_name = preprocessor.get_inputs()[3].name
mean_name = preprocessor.get_inputs()[4].name
var_name = preprocessor.get_inputs()[5].name
label_name = preprocessor.get_outputs()[0].name

pred_onx = preprocessor.run([label_name], {
    input_name: data.astype(np.float32),
    size_name: sizes.astype(np.int64),
    scale_name: scale.astype(np.float32),
    bias_name: bias.astype(np.float32),
    mean_name: mean.astype(np.float32),
    var_name: var.astype(np.float32),
})[0]

In [16]:
pred_onx.shape

(1, 3, 224, 224)

In [17]:
pred_onx

array([[[[ 0.74447197,  0.74447197,  0.74447197, ...,  5.081202  ,
           5.081202  ,  5.081202  ],
         [ 0.74447197,  0.74447197,  0.74447197, ...,  5.081202  ,
           5.081202  ,  5.081202  ],
         [ 0.74447197,  0.74447197,  0.74447197, ...,  5.081202  ,
           5.081202  ,  5.081202  ],
         ...,
         [ 0.74447197,  0.74447197,  0.74447197, ..., 16.645815  ,
          16.645815  , 16.645815  ],
         [ 0.74447197,  0.74447197,  0.74447197, ..., 16.645815  ,
          16.645815  , 16.645815  ],
         [ 0.74447197,  0.74447197,  0.74447197, ..., 16.645815  ,
          16.645815  , 16.645815  ]],

        [[ 0.7907458 ,  0.7907458 ,  0.7907458 , ...,  5.1514764 ,
           5.1514764 ,  5.1514764 ],
         [ 0.7907458 ,  0.7907458 ,  0.7907458 , ...,  5.1514764 ,
           5.1514764 ,  5.1514764 ],
         [ 0.7907458 ,  0.7907458 ,  0.7907458 , ...,  5.1514764 ,
           5.1514764 ,  5.1514764 ],
         ...,
         [ 0.7907458 ,  0.7907458 

## Merge preprocessor and the model

In [18]:
preporcessor = onnx.load('preprocessor.onnx')

In [6]:
resnet = onnx.load('resnet50-v1-12-int8.onnx')
resnet.opset_import

[domain: ""
version: 12
, domain: "com.microsoft"
version: 1
, domain: "com.microsoft.mlfeaturizers"
version: 1
, domain: "com.microsoft.nchwc"
version: 1
, domain: "ai.onnx.training"
version: 1
, domain: "ai.onnx.preview.training"
version: 1
, domain: "com.microsoft.experimental"
version: 1
, domain: "ai.onnx.ml"
version: 2
]

In [ ]:
converted_resnet = version_converter.convert_version(resnet, 12)
converted_resnet.opset_import

In [40]:
resnet.ir_version

4

In [41]:
resnet.opset_import[0].version = 14

In [42]:
resnet.ir_version = 7

In [43]:
combined_model = onnx.compose.merge_models(
    preporcessor, resnet,
    io_map=[('Y', 'data')]
)

In [44]:
onnx.save(combined_model, 'combined_resnet.onnx')